In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
from PIL import Image
import random
import warnings
from collections import Counter
warnings.filterwarnings("ignore")

In [6]:
import torch
print(torch.__version__)  # Check PyTorch version
print(torch.version.cuda)  # Check CUDA version PyTorch was built with

2.5.1+cu121
12.1


In [7]:
import torch
print(torch.__version__)  # Check PyTorch version
print(torch.version.cuda)  # Check CUDA version PyTorch was built with

2.5.1+cu121
12.1


In [8]:
!nvidia-smi

Wed Feb 12 00:58:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.16                 Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8              5W /   50W |    3815MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
# Define transformations (Resize, Normalize, Augment)
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [16]:
# Assuming dataset is a folder-based ImageFolder dataset
dataset = datasets.ImageFolder("thermal", transform=transform)

# Count occurrences of each class
class_counts = Counter([label for _, label in dataset.samples])
max_count = max(class_counts.values())  # Find the class with the highest count

print(class_counts)  # Print class distribution


Counter({0: 220, 3: 142, 2: 93, 5: 80, 1: 67, 4: 34})


In [17]:
# Split dataset into 60% Train, 20% Validation, 20% Test
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size  # Remaining samples go to test

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

In [18]:
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=0)

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve

def train_inception(train_loader, val_loader, num_classes=6, num_epochs=20, lr=0.0001):
    # Ensure GPU is available
    if not torch.cuda.is_available():
        raise RuntimeError("GPU not available. This code requires a GPU to run.")
    device = torch.device("cuda")
    print(f"\nUsing device: {device} ({torch.cuda.get_device_name(device)})\n")

    # Load pretrained Inception v3
    model = models.inception_v3(weights="DEFAULT")
    
    # Modify classifier for 6 classes (Inception has both primary and auxiliary classifiers)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    if model.AuxLogits:
        model.AuxLogits.fc = nn.Linear(model.AuxLogits.fc.in_features, num_classes)
    
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    print(f"\nTraining Inception v3 for {num_epochs} epochs...\n")
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs, aux_outputs = model(images)
            
            loss = criterion(outputs, labels)
            if model.AuxLogits:
                loss_aux = criterion(aux_outputs, labels)
                loss += 0.4 * loss_aux  # Auxiliary loss factor
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            progress_bar.set_postfix(loss=running_loss / len(train_loader), accuracy=100 * correct / total)
        
        print(f"Epoch {epoch+1}: Loss = {running_loss / len(train_loader):.4f}, Accuracy = {100 * correct / total:.2f}%")
    
    # Evaluation Function with PR Curve
    def evaluate_model(loader, dataset_type="Validation"):
        model.eval()
        correct, total = 0, 0
        y_true, y_pred, y_scores = [], [], []
        
        with torch.no_grad():
            for images, labels in tqdm(loader, desc=f"🔍 Evaluating {dataset_type} Set", leave=True):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                
                _, preds = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())
                y_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())
                
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        accuracy = 100 * correct / total
        print(f"\n{dataset_type} Set Accuracy: {accuracy:.2f}%")
        print(classification_report(y_true, y_pred))
        print(confusion_matrix(y_true, y_pred))
        
        # Precision-Recall Curve Plot
        y_true_one_hot = np.eye(num_classes)[y_true]
        plt.figure(figsize=(10, 6))
        for class_idx in range(num_classes):
            precision, recall, _ = precision_recall_curve(y_true_one_hot[:, class_idx], np.array(y_scores)[:, class_idx])
            plt.plot(recall, precision, label=f'Class {class_idx}')
        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'Precision-Recall Curve for {dataset_type} Set')
        plt.legend()
        plt.grid()
        plt.show()
    
    # Evaluate on Validation Set
    evaluate_model(val_loader, "Validation")
    evaluate_model(test_loader, "Testing")
    return model


In [ ]:
train_inception(train_loader, val_loader, num_classes=6, num_epochs=20, lr=0.0001)


Using device: cuda (NVIDIA GeForce GTX 1650)


Training Inception v3 for 20 epochs...



Epoch 1/20: 100%|████████████████████████████████████████████| 12/12 [02:12<00:00, 11.01s/it, accuracy=39.6, loss=2.23]


Epoch 1: Loss = 2.2269, Accuracy = 39.63%


Epoch 2/20: 100%|████████████████████████████████████████████| 12/12 [02:16<00:00, 11.34s/it, accuracy=67.5, loss=1.41]


Epoch 2: Loss = 1.4148, Accuracy = 67.45%


...

Epoch 19/20: 100%|█████████████████████████████████████████| 12/12 [02:13<00:00, 11.11s/it, accuracy=99.7, loss=0.0214]


Epoch 19: Loss = 0.0214, Accuracy = 99.74%


Epoch 20/20: 100%|██████████████████████████████████████████| 12/12 [02:13<00:00, 11.11s/it, accuracy=99.7, loss=0.026]


Epoch 20: Loss = 0.0260, Accuracy = 99.74%


🔍 Evaluating Validation Set:  50%|███████████████████████████                           | 2/4 [00:05<00:05,  2.73s/it]